In [34]:
import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt
from collections import Counter
import xgboost as xgb
from sklearn import cross_validation, grid_search, metrics, ensemble
from sklearn.preprocessing import  MinMaxScaler

C:\Users\a0970\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\a0970\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [11]:
#load data
df_train = pd.read_csv('..\\kaggle048\\train.csv',header=None)
df_test = pd.read_csv('..\\kaggle048\\test.csv',header=None)

In [15]:
print('train:',df_train.shape)
print('test',df_test.shape)

train: (1000, 40)
test (9000, 40)


In [20]:
#load label
label = pd.read_csv('..\\kaggle048\\trainLabels.csv',header = None)

In [22]:
#label

In [16]:
df_train.head()

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,0.299403,-1.226624,1.498425,-1.176150,5.289853,0.208297,2.404498,1.594506,-0.051608,0.663234,...,-0.850465,-0.622990,-1.833057,0.293024,3.552681,0.717611,3.305972,-2.715559,-2.682409,0.101050
1,-1.174176,0.332157,0.949919,-1.285328,2.199061,-0.151268,-0.427039,2.619246,-0.765884,-0.093780,...,-0.819750,0.012037,2.038836,0.468579,-0.517657,0.422326,0.803699,1.213219,1.382932,-1.817761
2,1.192222,-0.414371,0.067054,-2.233568,3.658881,0.089007,0.203439,-4.219054,-1.184919,-1.240310,...,-0.604501,0.750054,-3.360521,0.856988,-2.751451,-1.582735,1.672246,0.656438,-0.932473,2.987436
3,1.573270,-0.580318,-0.866332,-0.603812,3.125716,0.870321,-0.161992,4.499666,1.038741,-1.092716,...,1.022959,1.275598,-3.480110,-1.065252,2.153133,1.563539,2.767117,0.215748,0.619645,1.883397
4,-0.613071,-0.644204,1.112558,-0.032397,3.490142,-0.011935,1.443521,-4.290282,-1.761308,0.807652,...,0.513906,-1.803473,0.518579,-0.205029,-4.744566,-1.520015,1.830651,0.870772,-1.894609,0.408332


In [25]:
#use ntrain as index to get data back
ntrain = df_train.shape[0]
ntest = df_test.shape[0]
df = pd.concat((df_train,df_test)).reset_index(drop=True)

In [27]:
#missing data
total = df.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(5)
# no missing value

,Total,Percent
39,0,0.0
38,0,0.0
17,0,0.0
16,0,0.0
15,0,0.0


In [28]:
#normalization

MMEncoder = MinMaxScaler()
df= MMEncoder.fit_transform(df)

In [33]:
train = df[:ntrain]
test = df[ntrain:]

In [39]:
trains = pd.DataFrame(train)
tests = pd.DataFrame(test)

In [41]:
train_data, test_data, train_labels, test_labels = cross_validation.train_test_split(trains, label, test_size = 0.2)

In [43]:
model_xgb = xgb.XGBClassifier(n_estimators=300, max_depth=5, learning_rate=0.1,min_child_weight=10)

In [44]:
model_xgb.fit(train_data, train_labels)

C:\Users\a0970\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\a0970\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=10, missing=None, n_estimators=300,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [46]:
pred_xgb = model_xgb.predict(tests)

C:\Users\a0970\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [55]:
ID = list(range(1,9001))

In [56]:
test_ans = pd.DataFrame({'Id': ID, 'Solution': pred_xgb })

In [58]:
test_ans.to_csv('answer048_0802.csv',index=False)